<a href="https://colab.research.google.com/github/prasys/lightstay-automation/blob/main/Hilton_LightStay_Automation_Project_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pre-Installation - Installing Chrome

Here we will be installing Selenium and Chrome together. Selenium is a web automation tool to simulate click and a lot of other things. We will also be instlaling Chrome

In [ ]:
!pip install selenium
!pip install tqdm

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300



Executing: /tmp/apt-key-gpghome.pgUsw8CYGk/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.uiH1sqBPbp/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.rtkQUJdnOx/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://deb.debian.org/debian-security buster/updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 http://archive.ubuntu.com/ubuntu jammy-updates/universe 

#Import phases

Here we will be importing libarires and a lot of other configuration, please make sure that you change the filename `filename.csv` to something else.

We are fooling the system to have a look and thinking that it is a chrome browser running on a full screen on Linux. It's to trick the system that it is a genuine human with a real computer clicking it through!

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException  # Import TimeoutException
import time
import pandas as pd
from tqdm import tqdm
import datetime

## FILE CONFIGURATION - CURRENTLY THE FILENAME IS SET TO FILEOUT.CSV
df = pd.read_csv('march.csv')

#FOR OUR LOOKUP TABLE
df_lookup = pd.read_csv('lookup.csv')
#df_lookup['State'] = df_lookup['State'].replace('Fiji', 'Queensland')


# DATA CLEANING!!
df.columns = df.columns.str.strip()
progress_bar = tqdm(total=len(df), position=0, leave=True)
df = df.replace('-','0')
df.replace('-', 0, inplace=True)
df = df.replace(' -   ','0')
df = df.fillna(0)
df_lookup = df_lookup.fillna(0)
df['Meeting /Event Name'] = df['Meeting /Event Name'].str.replace('[^a-zA-Z0-9\s]', '', regex=True) # fix the names


  0%|          | 0/264 [05:15<?, ?it/s]


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--verbose")
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument("--window-size=1920, 1200")
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)


# Navigate to the desired webpage
driver.get('https://lightstay.ei3.com/')

In [ ]:
# Wait for the page to load (optional but recommended)
driver.implicitly_wait(10)  # wait up to 10 seconds

USER = "enteruser"
PASSWORD = "password"

# Locate the username and password fields
username_field = driver.find_element(By.ID, 'loginform:username')
password_field = driver.find_element(By.ID, 'loginform:password')
driver.execute_script("arguments[0].value='your_password';", password_field)



username_field.send_keys('cehmann')
driver.execute_script("arguments[0].value='adelaide';", password_field)
# Locate and click the login button
login_button = driver.find_element(By.ID, 'loginform:loginButton')
login_button.click()


#Phase 1 - Data Entry and Mark it as submitted

Here, we will be doing the following


*   Read the file (which contains hotel details) and the code
*   For each row, input the details on the fields and mark it as 'Submitted'
*   Save it

In [ ]:
for index, row in df.iterrows():
    driver.get('https://lightstay.ei3.com/property/' + str(row['Hotel Code']) + '/tools/meetingcalculator/new')
    driver.find_element(By.ID, 'meetCalcForm:meetingName').send_keys(row['Meeting /Event Name'])
    driver.find_element(By.ID, 'meetCalcForm:recipientCompany').send_keys(row['Client / Account Name'])
    driver.find_element(By.ID, 'meetCalcForm:recipientName').send_keys(row['Client / Account Contact'])
    driver.find_element(By.ID, 'meetCalcForm:hotelName').send_keys(row['Hotel Name'])
    driver.find_element(By.ID, 'meetCalcForm:rfpNumber').send_keys(row['RFP number']) # fix RFP Number
#    driver.find_element(By.ID, 'meetCalcForm:meetingStatus').send_keys(row['Meeting Status'])
    date_value = row['Meeting/Event Start date']
    print(row['Meeting /Event Name'])
    date_object = datetime.datetime.strptime(date_value, "%d/%m/%Y")
    formatted_date_value = date_object.strftime("%m/%d/%Y")
    js_script = "document.getElementById('meetCalcForm:eventDateInputDate').value = '{}';".format(formatted_date_value)
    driver.execute_script(js_script)
    driver.find_element(By.ID, 'meetCalcForm:numberGuestNights').send_keys(row['Room Block w meeting'])

    driver.find_element(By.ID, 'meetCalcForm:onPeakNights').send_keys(row['Number of room nights booked on peak night w meeting'])
    driver.find_element(By.ID, 'meetCalcForm:meetingFloorSpace').send_keys(row['Meeting/Event Floor Space'])
    driver.find_element(By.ID, 'meetCalcForm:meetingDuration').send_keys(row['Meeting room Duration'])
    attendees_value = row['Number of atttendees']
    js_script = f"document.getElementById('meetCalcForm:numberGuests').value = '{attendees_value}';"
    driver.execute_script(js_script)
    brekkie_value = row['Breakfast']
    js_script = f"document.getElementById('meetCalcForm:breakfasts').value = '{brekkie_value}';"
    driver.execute_script(js_script)
    lunch_value = row['Lunch']
    js_script = f"document.getElementById('meetCalcForm:lunches').value = '{lunch_value}';"
    driver.execute_script(js_script)
    dinner_value = row['Dinner']
    js_script = f"document.getElementById('meetCalcForm:dinners').value = '{dinner_value}';"
    driver.execute_script(js_script)
    snacks_value = row['Break']
    js_script = f"document.getElementById('meetCalcForm:snacks').value = '{snacks_value}';"
    driver.execute_script(js_script)
    # driver.find_element(By.ID, 'meetCalcForm:hotelContactName').send_keys(row['Hotel Contact Name'])
    # driver.find_element(By.ID, 'meetCalcForm:hotelNameCoverPage').send_keys(row['Hotel Name (Cover Page)'])
    # driver.find_element(By.ID, 'meetCalcForm:hotelContactEmail').send_keys(row['Hotel Contact Email'])
    # driver.find_element(By.ID, 'meetCalcForm:hotelContactPhone').send_keys(row['Hotel Contact Telephone Number'])
    # driver.find_element(By.ID, 'meetCalcForm:titlePageCheckbox').send_keys(row['Title Page'])
    # driver.find_element(By.ID, 'meetCalcForm:reportUnits').send_keys(row['Report Units'])
    #force few things
    driver.execute_script("document.getElementById('meetCalcForm:spaceUnitsselValue').value = 'm2';")
    driver.execute_script("document.getElementById('meetCalcForm:spaceUnitsInput').value = 'm²';")
    driver.execute_script("""
        document.getElementById('meetCalcForm:meetingStatusselValue').value = '1';
        document.getElementById('meetCalcForm:meetingStatusInput').value = 'Submitted';
    """)
    driver.execute_script("document.getElementById('meetCalcForm:durationTypeselValue').value = 'hours';")
    driver.execute_script("document.getElementById('meetCalcForm:durationTypeInput').value = 'Hours';")
    # Submit the form after filling it out (assuming there's a submit button)
    element = driver.find_element(By.ID, 'meetCalcForm:saveReportButton')
    driver.execute_script("arguments[0].click();", element)
    current_url = driver.current_url
    expected_url = 'https://lightstay.ei3.com/property/' + str(row['Hotel Code']) + '/tools/meetingcalculator/submitconfirm'
    if current_url == expected_url:
      print("Entered Correctly!")
      progress_bar.update()
    else:
      print(f"Unexpected URL: {current_url}")
      print(row['Meeting /Event Name'])

    # Optionally, add a delay to avoid overwhelming the server
    time.sleep(1)



3263  NVS NCCMEDICAL


  0%|          | 1/503 [00:13<1:50:02, 13.15s/it]

Entered Correctly!
TRANSFORM 2024


  0%|          | 2/503 [00:16<1:01:01,  7.31s/it]

Entered Correctly!
LawAustralasia Workshop 2024


...

Entered Correctly!
AFAP Meeting


 98%|█████████▊| 495/503 [28:04<00:26,  3.36s/it]

Entered Correctly!
Martin MacDonald Seminar


 99%|█████████▊| 496/503 [28:07<00:23,  3.31s/it]

Entered Correctly!


#Phase 2 - Mark Status as Completed

Here, we will be doing the following


*   Read the file (which contains hotel details)
*   Do a search (based on the criteria 'Submitted') and the name
*   Select the first element and click on the link
*   Mark it as Completed




In [ ]:
# Assuming 'driver' is already defined and imported along with other necessary modules

progress_bar = tqdm(total=len(df), position=0, leave=True)
failed_meetings = []

for index, row in df.iterrows():
    try:
        print(str(row['Hotel Code']))
        driver.get('https://lightstay.ei3.com/property/' + str(row['Hotel Code']) + '/tools/meetingcalculator?statusFilter=1')

        # Ensure 'Submitted' is checked
        submitted_checkbox = driver.find_element(By.ID, 'meetingForm:statusFilter:1')
        if not submitted_checkbox.is_selected():
            submitted_checkbox.click()

        # Enter the meeting/event name in the search box
        search_box = driver.find_element(By.ID, 'meetingForm:txtFilterText')
        search_box.clear()
        search_box.send_keys(row['Meeting /Event Name'])
        print(row['Meeting /Event Name'])
        search_box.send_keys(Keys.RETURN)
        driver.implicitly_wait(2)

        first_row = driver.find_element(By.ID, 'meetingForm:meetingTable:0')

        # Processing the first row
        meeting_name_link = first_row.find_element(By.XPATH, './/a[contains(@onclick, "jsf.util.chain")]')
        driver.execute_script("arguments[0].click();", meeting_name_link)

        # Inject code
        driver.execute_script("""
            document.getElementById('meetCalcForm:meetingStatusselValue').value = '2';
            document.getElementById('meetCalcForm:meetingStatusInput').value = 'Completed';
        """)

        element = driver.find_element(By.ID, 'meetCalcForm:saveReportButton')
        driver.execute_script("arguments[0].click();", element)
        time.sleep(0.5)
    except Exception as e:
        print(f"Failed to process meeting: {row['Meeting /Event Name']} due to error: {e}")
        failed_meetings.append(row['Meeting /Event Name'])

    progress_bar.update()

progress_bar.close()

# Print the list of failed meetings
print("Failed Meetings:")
for meeting in failed_meetings:
    print(meeting)

# Consider closing the driver at the end of the process
# driver.close()

 99%|█████████▊| 496/503 [28:27<00:24,  3.44s/it]


SYDHI
3263  NVS NCCMEDICAL


  0%|          | 1/503 [00:04<36:11,  4.33s/it]

SYDHI
TRANSFORM 2024


  0%|          | 2/503 [00:09<38:34,  4.62s/it]

...

PERDT
AFAP Meeting


100%|█████████▉| 502/503 [31:52<00:03,  3.57s/it]

PERDT
Martin MacDonald Seminar


100%|██████████| 503/503 [31:57<00:00,  3.81s/it]

Failed Meetings:
ANZAOMS Meeting
ECU Team Building Event
FLS Training
Fulton Hogan March 2024 Meeting
PNGICSA SPECIAL ADVISORY BOARD MEETING
APT  Travelmarvel Presentations  19th March 2024
AMOSC Meeting 12th March 2024
Cassandra  Nathan Wedding March 2024
EMS Oceania Meeting  9th March 2024
USAID Conference 6th March 2024
USAID Conference 27th Feb 1st March 2024
USAID Conference 4th March 2024
USAID Conference 8th March 2024
Rio Tinto Recruitment  14th March 2024
ASTEC Meeting
ISEP Study Abroad  Breakfast  5th March 2024
AREEA  Corporate Breakfast  14th March 2024
Chanel  Meeting 5th March 2024
Powering WA Planning Sessions  15th March 2024
Powering WA Planning Sessions  13th March 2024
Netwealth Investments  Product Roadshow  21st March 2024
Bali Hotels Association  Perth Event  14th March 2024
BHP  Conference 15th March 2024
Rio Tinto Recruitment  20th March 2024
Rio Tinto Recruitment  27th March 2024
Cash Converters Meeting  12th March 2024


# Phase 3

In [ ]:
failed_meetings = []
df_lookup = pd.read_csv('lookup.csv')

progress_bar = tqdm(total=len(df), position=0, leave=True)
#special cleaning
regex = r'.*[-@!\'](.*)'
extracted = df['Meeting /Event Name'].str.extract(regex, expand=False)
df['Meeting /Event Name'] = extracted.fillna(df['Meeting /Event Name'])

#merge our df to include additional details, so we can easily fill in the rows/columns


# Merge the dataframes based on 'id'
result = df.merge(df_lookup, on='Hotel Code', how='left')


for index, row in result.iterrows():
  print(str(row['Hotel Code']) + " " + str(row['Meeting /Event Name']))
  driver.get('https://lightstay.ei3.com/property/' + str(row['Hotel Code']) + '/tools/meetingcalculator?statusFilter=2')
  #page_source = driver.page_source

# Print the page source
 # print(page_source)



  # Ensure 'Submitted' is checked
  try:
    submitted_checkbox = driver.find_element(By.ID, 'meetingForm:statusFilter:2')
    if not submitted_checkbox.is_selected():
        submitted_checkbox.click()  # This will check the 'Submitted' checkbox if it isn't already checked
  except Exception as e:
      print(f"Step 1 Failed: {row['Meeting /Event Name']} due to error! SKIPPING")
      failed_meetings.append(row['Meeting /Event Name'])
      continue

  # Enter the meeting/event name in the search box
  search_box = driver.find_element(By.ID, 'meetingForm:txtFilterText')
  search_box.clear()  # Clear any existing text

  search_box.send_keys(row['Meeting /Event Name'])

  # Trigger the search by pressing Enter
  search_box.send_keys(Keys.RETURN) # Stimulate Enter and wait
  driver.implicitly_wait(10) # wait for 2 seconds

  try:
    element = driver.find_element(By.XPATH, "//input[@id='meetingForm:meetingTable:0:meetingActionSelectInput']")
    driver.execute_script("arguments[0].click();", element)

    element = driver.find_element(By.XPATH, "//div[@id='meetingForm:meetingTable:0:meetingActionSelectItem1']")
    driver.execute_script("arguments[0].click();", element)

    wait = WebDriverWait(driver, timeout=10)
    iframe_element = wait.until(EC.presence_of_element_located((By.ID, 'climeCoForm')))
    driver.switch_to.frame(iframe_element)

  # element = driver.find_element(By.XPATH, "//input[@id='meetingForm:meetingTable:0:meetingActionSelectInput']")
  # driver.execute_script("arguments[0].click();", element)
 #   try:
 #     element = driver.find_element(By.XPATH,"//div[@class='cky-consent-bar']")
 #     accept_button = driver.find_element_by_xpath("//button[contains(@class, 'cky-btn-accept')]")
 #     driver.execute_script("arguments[0].click();", accept_button)
    #  accept_button.click()


#    except Exception as e:
#      print("Not found - continue")


   # checkout_button = driver.find_element(By.XPATH, "//a[contains(text(),'Checkout Now')]")
  #  checkout_button.click()
  except Exception as e:
        print(f"Step 2 Failed : {row['Meeting /Event Name']} due to error! SKIPPING")
        failed_meetings.append(row['Meeting /Event Name'])
        continue

  try:
      wait = WebDriverWait(driver, 10)
      accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept All')]")))
      accept_button.click()
      print("Cookie consent accepted.")
  except Exception as e:
      print(f"Error accepting cookies: {e}")

  checkout_button = driver.find_element(By.XPATH, "//a[contains(text(),'Checkout Now')]")
  checkout_button.click()

  # this is where, we will populate the things for our form


  driver.implicitly_wait(10)

  try:
      wait = WebDriverWait(driver, 10)
      accept_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept All')]")))
      accept_button.click()
      print("Cookie consent accepted.")
  except Exception as e:
      print(f"Error accepting cookies: {e}")


  driver.find_element(By.ID, 'billing_first_name').clear()
  driver.find_element(By.ID, 'billing_first_name').send_keys(row['Hotel Code'])
 # district = driver.find_element(By.ID, "billing_first_name")
 # print(district.get_attribute("value"))
  driver.find_element(By.ID, 'billing_last_name').clear()
  driver.find_element(By.ID, 'billing_last_name').send_keys(row['Hotel Code'])
  driver.find_element(By.ID, 'billing_company').clear()
  driver.find_element(By.ID, 'billing_company').send_keys(row['Company'])



  country_name = row['Coiuntry/Region']
  select2_element = driver.find_element(By.CSS_SELECTOR, ".select2-selection")
  select2_element.click()

  # Wait for the desired option to be visible, then click it
  desired_option_xpath = f"//li[text()='{country_name}']"
  desired_option = wait.until(EC.visibility_of_element_located((By.XPATH, desired_option_xpath)))
  desired_option.click()
  wait = WebDriverWait(driver, timeout=10)
  iframe_element = wait.until(EC.presence_of_element_located((By.ID, 'billing_city')))

  driver.find_element(By.ID, 'billing_address_1').clear()
  driver.find_element(By.ID, 'billing_address_1').send_keys(row['Street address'])
  # add a check here to determine if it's Australia or New Zealand



  driver.find_element(By.ID, 'billing_city').clear()
  driver.find_element(By.ID, 'billing_city').send_keys(row['Suburb / City'])
  try:
    select2_container = driver.find_element(By.ID, "select2-billing_state-container")
    select2_container.click()
  except Exception as e:
      print("Error -Skipping")

  state_name = row['State']

  desired_option_xpath = f"//li[text()='{state_name}']"
  driver.implicitly_wait(20)
  try:
   desired_option = wait.until(EC.visibility_of_element_located((By.XPATH, desired_option_xpath)))
   desired_option.click()
  except TimeoutException as e:
    print(f"Timeout while waiting for element: {desired_option_xpath}")
    print(f"Error message: {e}")
  wait = WebDriverWait(driver, timeout=50)
  iframe_element = wait.until(EC.presence_of_element_located((By.ID, 'billing_postcode')))


  driver.find_element(By.ID, 'billing_postcode').clear()
  driver.find_element(By.ID, 'billing_postcode').send_keys(str(int(row['Postcode'])))
  driver.find_element(By.ID, 'billing_phone').clear()
  driver.find_element(By.ID, 'billing_phone').send_keys(str(int(row['Phone'])))
  driver.find_element(By.ID, 'billing_email').clear()
  driver.find_element(By.ID, 'billing_email').send_keys(str(row['email'].strip()))
  driver.find_element(By.ID, 'billing_city').clear()
  driver.find_element(By.ID, 'billing_city').send_keys(row['Suburb / City'])

 # element = driver.find_element(By.XPATH,"//div[@class='cky-consent-bar']")
 # accept_button = driver.find_element(By.XPATH, "//button[contains(@class, 'cky-btn-accept')]")
 # driver.execute_script("arguments[0].click();", accept_button)
  #driver.find_element(By.ID, 'billing_email').send_keys(str('test@test.com'))

  try:
    #WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "affiliate_consent"))).click()
    checkbox = driver.find_element(By.ID, "affiliate_consent")
    checkbox.click()
    # driver.execute_script("arguments[0].click();", checkbox)
  except Exception as e:
      print("Error")
      print("Retrying")
      wait = WebDriverWait(driver, 10)
      checkbox = wait.until(EC.element_to_be_clickable((By.ID, "affiliate_consent")))
      driver.execute_script("arguments[0].click();", checkbox)

     # checkbox = driver.find_element(By.ID, "affiliate_consent")
     # checkbox.click()
      #driver.execute_script("arguments[0].click();", checkbox)

  #checkbox = driver.find_element(By.ID, "affiliate_consent")
  #checkbox.click()

 # except Exception as e:
 #   failed_meetings.append(row['Meeting /Event Name'])
 #   continue


  place_order_button = driver.find_element(By.ID, "place_order")
  driver.execute_script("arguments[0].click();", place_order_button)
  driver.implicitly_wait(5) # wait for 5 seconds

  # try:
  #   # Find the element by class name
  #   error_element = driver.find_element(By.CLASS_NAME, 'woocommerce-error')

  #   # Print the text content of the element
  #   print(error_element.text)
  #   print(f"Step 3 Failed: {row['Meeting /Event Name']} due to error: {e}")
  #   failed_meetings.append(row['Meeting /Event Name'])
  #   continue
  # except Exception as e:
  #   print("No error found continue")
  #   continue


  try:
      # Wait for error messages to be present in the DOM
      wait = WebDriverWait(driver, 10)
      error_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'woocommerce-error')))

      # Collect all error messages
      errors = [error.text for error in error_elements]
      error_message = "\n".join(errors)

      # Log the error message
      if error_message:
          print("Errors encountered:")
          print(error_message)
          print(f"Step 3 Failed: {row['Meeting /Event Name']} due to error: {error_message}")
          failed_meetings.append(row['Meeting /Event Name'])
      else:
          print("No errors found, but exception was raised.")
  except Exception as e:
      print(f"No error found continue: {str(e)}")
      continue





  try:
    element = WebDriverWait(driver, 10).until(
        EC.text_to_be_present_in_element((By.ID, "mb-10"), "Thank you! Your funds have been received!")
    )
    print("Text is present!")
  except TimeoutException:
    print("Text is not present or page did not load within 10 seconds.")

  driver.switch_to.default_content() # switch back

  driver.execute_script('document.getElementById("meetPurchaseOffsetsForm:carbonOffsetsPurchased").click();')


progress_bar.close()
#driver.close()


  0%|          | 0/258 [00:11<?, ?it/s]


SYDHI RMIT University Workshop
Cookie consent accepted.
Cookie consent accepted.
Error
Retrying
No error found continue: Message: 

SYDHI Ikon Institute of Australia Graduation 2024
Cookie consent accepted.
Cookie consent accepted.
No error found continue: Message: 

SYDHI Tamara Meyer Coaching
Cookie consent accepted.
Cookie consent accepted.
No error found continue: Message: 

CHCNZ Canterbury March 2024 Ballance Farm Environment Awards
Cookie consent accepted.
Cookie consent accepted.
Timeout while waiting for element: //li[text()='Fiji']
Error message: Message: 
Stacktrace:
#0 0x5853b83fe7f9 <unknown>
#1 0x5853b839e3b3 <unknown>
#2 0x5853b80e6016 <unknown>
#3 0x5853b811a81e <unknown>
#4 0x5853b81508fb <unknown>
#5 0x5853b813dded <unknown>
#6 0x5853b814e9e1 <unknown>
#7 0x5853b813dc93 <unknown>
#8 0x5853b810fce4 <unknown>
#9 0x5853b81114d2 <unknown>
#10 0x5853b83ca542 <unknown>
#11 0x5853b83d9ce7 <unknown>
#12 0x5853b83d99e4 <unknown>
#13 0x5853b83de13a <unknown>
#14 0x5853b83da5b9 

KeyboardInterrupt: 